In [17]:
wordlist = ['apple', 'tombs', 'heron', 'grape', 'melon']
#for x in (True):#, False):
#print(x)
rounds = 20000
random.seed(655765355)
with open('combined_wordlist_2.txt', 'r') as f:
    wordlist = f.read().split('\n')[1:]
player = AutoWordlePlayer(wordlist, rounds, WordleSolver, True)
player.play_round()
average_guesses = player.average_guesses()
print(f"Average guesses per round: {average_guesses}")
average_time = player.average_time()
print(f"Average time per round: {average_time}")
print(player.return_stats())
#for kl in player.history_history:
    #print(kl)
#three_pointers = player.threes

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
18

TypeError: unsupported operand type(s) for +=: 'set' and 'set'

In [18]:
three_pointers = player.threes

In [19]:
len(three_pointers)

1822

In [13]:
import random
from time import perf_counter_ns as pcn
class AutoWordlePlayer:
    def __init__(self, wordlist, n, agent, preguess):
        self.wordlist = wordlist
        self.current_word = random.choice(wordlist)
        self.rounds_played = 0
        self.total_guesses = 0
        self.feedback_history = []
        self.times_per_round = []
        self.n = n
        self.solving_agent = agent
        self.stats = {i:0 for i in range(20)}
        self.preguess = preguess
        self.history_history = []
        self.threes = set()

    def enter_playmode(self):
        print(f"Entering play mode with word: {self.current_word}")

    def make_guess(self, guess):
        feedback = self._get_feedback(guess)
        self.total_guesses += 1
        self.feedback_history.append((guess, feedback))
        return (guess, feedback)

    def _get_feedback(self, guess):
        feedback = ''
        for g, w in zip(guess, self.current_word):
            if g == w:
                feedback += 'g'
            elif g in self.current_word:
                feedback += 'y'
            else:
                feedback += 'r'
        return feedback

    def play_round(self):
        for i in range(self.n):
            if i%100==0:
                print(i)
            self.feedback_history = []
            self.solver = self.solving_agent(True, self.preguess)
            self.current_word = random.choice(wordlist)
            start = pcn()
            round_guesses = 0
            while True:
                #feedback = ''.join(fb[1] for fb in self.feedback_history)
                guess = self.solver.guess(self.feedback_history)
                feedback = self.make_guess(guess)
                #print(guess, feedback)
                #print(f"{guess} {feedback[1]}")
                round_guesses += 1
                if feedback[1] == 'ggggg':
                    self.stats[round_guesses] += 1
                    self.times_per_round.append(pcn()-start)
                    self.rounds_played += 1
                    if round_guesses >= 6:
                        self.history_history.append(self.feedback_history.copy())
                    if round_guesses == 3:
                        self.threes.add(guess)
                    break
            #print([x[0] for x in self.feedback_history])
                
                
                
    def return_stats(self):
        deled = []
        for k,v in self.stats.items():
            if not v:
                deled.append(k)
        for x in deled:
            del self.stats[x]
        return self.stats
    def average_guesses(self):
        if self.rounds_played == 0:
            return 0
        return self.total_guesses / self.rounds_played
    def average_time(self):
        if self.rounds_played == 0:
            return 0
        return sum(self.times_per_round) / self.rounds_played / 1_000_000_000


In [9]:
from collections import Counter
class WordleSolver:
    with open('combined_wordlist_2.txt', 'r') as f:
        read_wordlist = f.read().split('\n')[1:]
        all_words = {c: {i: set() for i in range(5)} for c in 'abcdefghijklmnopqrstuvwxyz'}
        for c in 'abcdefghijklmnopqrstuvwxyz':
            all_words[c]['all'] = set()
        zzz = set()
        for word in read_wordlist:
            for idx,c in enumerate(word):
                all_words[c][idx].add(word)
                all_words[c]['all'].add(word)
        possible_words = set()
        for c in 'abcdefghijklmnopqrstuvwxyz':
            possible_words |= all_words[c]['all'].copy()

    def __init__(self, hard_mode=True, preguess=False):
        self.letters = dict()
        self.letters['g'] = {i: set() for i in range(5)}
        self.letters['y'] = {i: set() for i in range(5)}
        self.letters['r'] = {i: set() for i in range(5)}
        self.possible_words = set()
        self.green_words = set()
        self.yellow_words = WordleSolver.possible_words.copy()
        self.red_words = set()
        self.hard_mode = hard_mode
        self.guess_number = 0
        self.guesses = -1
        self.preguess = preguess
        #self.judge = WordleJudge()
       # self.get_all_words()

    def guess(self, history):
        if history:
            word, feedback = history[-1]
            for ch, fb, idx in zip(word, feedback, range(5)):
                if fb == 'g':
                    self.update_green_words(ch, idx)
                    pass
                elif fb == 'y':
                    self.update_yellow_words(ch, idx)
                    pass
                else:
                    self.update_red_words(ch, idx)
                    pass
        for unpack in history:
            word, feedback = unpack
            for w,f,i in zip(word, feedback, range(5)):
                self.letters[f][i].add(w)
        if self.preguess:
            if self.guesses < 1:
                self.guesses += 1
                return [('toeas', 'rrrrr'), ('nirly', 'rrrrr'), ('bumph', 'rrrrr')][self.guesses][0]
        
        if self.green_words:
            if self.hard_mode:
                final_words = (self.yellow_words & self.green_words) - self.red_words
            else:
                final_words = (self.yellow_words - self.green_words) - self.red_words
        else:
            final_words = self.yellow_words - self.red_words
        if not self.hard_mode:
            final_words = WordleSolver.possible_words.copy() - self.yellow_words - self.green_words - self.red_words
            self.guesses += 1
            if self.guesses >= 2:
                self.hard_mode = True

        out = self.analysis_hard_mode(final_words)
        if len(out) == 1:
            return out[0][0]
        outlist = [c[0] for c in out[:10]]
        #for k,v in {word: f'{self.judge.is_wordle_probability(word):.4}' for word in outlist}.items():
            #print(k, v)
        return sorted(outlist, reverse=True, key=lambda x: sum(self.ranked_word[x]))[0]
    
    def update_green_words(self, letter, position):
        if self.green_words:
            self.green_words &= WordleSolver.all_words[letter][position].copy()
        else:
            self.green_words = WordleSolver.all_words[letter][position].copy()
    
    def update_yellow_words(self, letter, position):
        if self.green_words:
            self.yellow_words &= self.green_words
        self.yellow_words &= (WordleSolver.all_words[letter]['all'].copy() - WordleSolver.all_words[letter][position].copy())

    def update_red_words(self, letter, position):
        self.red_words |= WordleSolver.all_words[letter]['all'].copy()
    

    def analysis_hard_mode(self, words):
        breakdown = Counter(''.join(words))
        dictio = {word: sum([breakdown[c] for c in set(word)]) for word in words}
        sorted_words = sorted([(k,v) for k,v in dictio.items()], reverse=True, key=lambda x: x[1])
        self.analysis(words)
        return sorted_words

    def analysis(self, words):
        ranking = {i: '' for i in range(5)}
        for word in words:
            for idx, c in enumerate(word):
                ranking[idx] += c
        for k,v in ranking.items():
            ranking[k] = Counter(v)
        self.ranked_word = {word: [ranking[idx][c] for idx, c in enumerate(word)] for word in words}
        #return sorted(words, reverse=True, key=lambda x: sum(self.ranked_word[x]))

        
        

In [5]:
import string


class WordList:
    """
    Reads a list of words from a file and filters it for wordle compatible words.
    Used to provide word list copies.
    """

    def __init__(self, filename):
        self.ascii_lowercase = list(string.ascii_lowercase)
        with open(filename, encoding="utf8") as file:
            self.words = file.readlines()
            self.words = [word.rstrip() for word in self.words]
            self.words = [w for w in self.words if len(w) == 5 and self.is_ascii_lowercase(w)]
            self.words = list(dict.fromkeys(self.words))  # remove duplicates


    def get_list_copy(self):
        return self.words.copy()

    def is_ascii_lowercase(self, word):
        for letter in word:
            if letter not in self.ascii_lowercase:
                return False
        return True
class WordleJudge:
    """
    Helper class to take into account how common words are in the English language.
    """

    def __init__(self, words=WordList("combined_wordlist.txt").words,
                 common_words=WordList("common_words.txt").words):
        self.common_words = common_words
        self.probability = {}
        for word in words:
            self.probability[word] = self.__calculate_probability(word)

    def __calculate_probability(self, word):
        if word not in self.common_words:
            return 368 / (368 + 8890)
        relative_position = self.common_words.index(word) / len(self.common_words)
        return 0.85 * (1 - relative_position) + 0.35 * relative_position

    def is_wordle_probability(self, word):
        """
        :param word: a 5 letter word
        :return: the probability of the word being a wordle based on its popularity in the English language
        """
        return self.probability[word]

In [6]:
t = WordleJudge()

In [36]:
from itertools import combinations_with_replacement as cwr
from itertools import permutations as perm
from itertools import cycle

In [61]:
poss = {(word, feedback): '' for word, feedback in zip(cycle(['aloes']), [''.join(combo) for combo in cwr('ygr', r=5)])}

In [64]:
poss

{('aloes', 'yyyyy'): '',
 ('aloes', 'yyyyg'): '',
 ('aloes', 'yyyyr'): 'loave',
 ('aloes', 'yyygg'): '',
 ('aloes', 'yyygr'): '',
 ('aloes', 'yyyrr'): 'coral',
 ('aloes', 'yyggg'): '',
 ('aloes', 'yyggr'): '',
 ('aloes', 'yygrr'): 'moola',
 ('aloes', 'yyrrr'): 'lairy',
 ('aloes', 'ygggg'): '',
 ('aloes', 'ygggr'): '',
 ('aloes', 'yggrr'): 'float',
 ('aloes', 'ygrrr'): '',
 ('aloes', 'yrrrr'): '',
 ('aloes', 'ggggg'): '',
 ('aloes', 'ggggr'): '',
 ('aloes', 'gggrr'): '',
 ('aloes', 'ggrrr'): '',
 ('aloes', 'grrrr'): '',
 ('aloes', 'rrrrr'): ''}

In [62]:
i = 0

In [63]:
for k,v in poss.items():
    try:
        g = WordleSolver(True, False)
        next_word1 = g.guess([])
        next_word2 = g.guess([k])
        print(next_word1, next_word2)
        poss[k] = next_word2
        i += 1
        if i >= 5:
            break
    except:
        print(k)

('aloes', 'yyyyy')
('aloes', 'yyyyg')
aloes loave
('aloes', 'yyygg')
('aloes', 'yyygr')
aloes coral
('aloes', 'yyggg')
('aloes', 'yyggr')
aloes moola
aloes lairy
('aloes', 'ygggg')
('aloes', 'ygggr')
aloes float


In [7]:
t.probability

{'noles': 0.03974940591920501,
 'melas': 0.03974940591920501,
 'waist': 0.7803628155826785,
 'ohias': 0.03974940591920501,
 'mummy': 0.6606503929108845,
 'shakt': 0.03974940591920501,
 'plume': 0.7364738338070557,
 'azurn': 0.03974940591920501,
 'aiyee': 0.03974940591920501,
 'rewth': 0.03974940591920501,
 'linum': 0.03974940591920501,
 'jembe': 0.03974940591920501,
 'porch': 0.7795268349774285,
 'moron': 0.03974940591920501,
 'vaute': 0.03974940591920501,
 'wahoo': 0.03974940591920501,
 'raile': 0.03974940591920501,
 'gimpy': 0.03974940591920501,
 'linac': 0.03974940591920501,
 'gyoza': 0.03974940591920501,
 'sword': 0.829685671292426,
 'cooey': 0.03974940591920501,
 'knurs': 0.03974940591920501,
 'safes': 0.03974940591920501,
 'mites': 0.4789918073900685,
 'eathe': 0.03974940591920501,
 'easle': 0.03974940591920501,
 'zeals': 0.03974940591920501,
 'bunjy': 0.03974940591920501,
 'claes': 0.03974940591920501,
 'alkos': 0.03974940591920501,
 'boxed': 0.6011285738170874,
 'lammy': 0.0397